<a target="_blank" href="https://colab.research.google.com/github/TransformerLensOrg/TransformerLens/blob/main/demos/BERT.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# BERT in TransformerLens
This demo shows how to use BERT in TransformerLens for the Masked Language Modelling task.

# Setup
(No need to read)

In [1]:
# NBVAL_IGNORE_OUTPUT
import os

# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = False
IN_GITHUB = os.getenv("GITHUB_ACTIONS") == "true"
try:
    import google.colab

    IN_COLAB = True
    print("Running as a Colab notebook")

    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/TransformerLensOrg/PySvelte.git
except:
    IN_COLAB = False

if not IN_GITHUB and not IN_COLAB:
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

if IN_COLAB:
    %pip install transformer_lens
    %pip install circuitsvis

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEVELOPMENT_MODE:
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "notebook_connected"
print(f"Using renderer: {pio.renderers.default}")

Using renderer: colab


In [3]:
import circuitsvis as cv

# Testing that the library works
cv.examples.hello("Neel")

In [4]:
# Import stuff
import torch

from transformers import AutoTokenizer

from transformer_lens import HookedEncoder

In [5]:
torch.set_grad_enabled(False)

# BERT

In this section, we will load a pretrained BERT model and use it for the Masked Language Modelling task

In [6]:
# NBVAL_IGNORE_OUTPUT
bert = HookedEncoder.from_pretrained("bert-base-cased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

If using BERT for interpretability research, keep in mind that BERT has some significant architectural differences to GPT. For example, LayerNorms are applied *after* the attention and MLP components, meaning that the last LayerNorm in a block cannot be folded.


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Moving model to device:  cpu
Loaded pretrained model bert-base-cased into HookedTransformer


Use the "[MASK]" token to mask any tokens which you would like the model to predict.

In [7]:
prompt = "BERT: Pre-training of Deep Bidirectional [MASK] for Language Understanding"

input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
mask_index = (input_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()

In [8]:
logprobs = bert(input_ids)[input_ids == tokenizer.mask_token_id].log_softmax(dim=-1)
prediction = tokenizer.decode(logprobs.argmax(dim=-1).item())

print(f"Prompt: {prompt}")
print(f'Prediction: "{prediction}"')

Prompt: BERT: Pre-training of Deep Bidirectional [MASK] for Language Understanding
Prediction: "Systems"


Better luck next time, BERT.